In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

year=[] 
for i in range(1997,2018):
    year.append(i)

all_url=[]
for x in year:
   all_url.append('https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(x))

#Create function to scrape URL 
def urlScraping(url):
    
    nba=BeautifulSoup(urlopen(url))

    #Obtain Headers
    headers=[th.getText() for th in nba.findAll('tr')[0].findAll('th')]
    headers=headers[1:]

    #Obtain player data in rows
    rows = nba.findAll('tr')[1:]
    player_stats=[[td.getText() for td in rows[i].findAll('td')] for i in range (len(rows))]

    #Obtain playerID from rows
    player_id=[]
    player=[[data['data-append-csv'] for data in rows[i].findAll('td',{'data-append-csv':True})] for i in range(len(rows))]

    for x in player:
        for y in x:
            player_id.append(y)


    #Create stats data frame
    st=pd.DataFrame(player_stats, columns=headers)

    col= [c for c in st.columns if c.isspace()==False]

    st=st[col]

    st=st[st['Player'].notnull()]

    st.reset_index(inplace=True)

    #Add Year to DataFrame
    yr = [x.getText() for x in nba.findAll('span')]
    r=re.compile(r'\d')
    yr_list=list(filter(r.match, yr))
    nba_year=int(re.findall('\d+', yr_list[0])[0])
    nba_year=nba_year+1

    st['Year']=nba_year


    #Add PlayerID to stats
    st['player_id']=player_id

    #Drop index column
    st.drop('index', axis=1, inplace=True)

    #Convert objects to floats/int
    st[['G', 'MP', 'PER', 'TS%', '3PAr', 'FTr','ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS','DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]=st[['G', 'MP', 'PER', 'TS%', '3PAr', 'FTr','ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS','DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']].apply(pd.to_numeric)
    
    return st

In [3]:
#Loop to create CSV Files
#for url in all_url:
#    data = urlScraping(url)
#   name = 'data/nba_advancedstats_'+str(data['Year'][0])+'.csv'
#    data.to_csv(name)
    

In [4]:
#Concat nba_advanced_stats data
file_name='data/nba_advancedstats_{}.csv'
adv_stats_list=[]
for x in range(1997,2018):
    adv_stats_list.append(pd.read_csv(file_name.format(x)))

adv_stats=pd.concat(adv_stats_list)


In [5]:
#To get salary data, the URL needs to be setup

player_id=[x for x in adv_stats['player_id'].unique()]
first_letter = []
for x in player_id:
    y=x[0]
    first_letter.append(y)

tup=list(zip(first_letter,player_id))

salary_url=[]
for x in tup:
    salary_url.append('https://www.basketball-reference.com/players/{}/{}.html'.format(*x))
    
#Create a function
def salScraping(url):
    html=requests.get(url)
    html_doc=html.text.replace('<!--','').replace('-->','')
    content=BeautifulSoup(html_doc,'html.parser')
    sal=content.find(id='all_salaries')
    id=content.find('meta',attrs={'property':'og:url'})
    pl_id=re.findall(r'/www.basketball-reference.com/players/[a-z]/(.*).html', str(id))
    header=[th.getText() for th in sal.findAll('tr')[0].findAll('th')]
    header=header[1:]
    rows=sal.findAll('tr')[1:]
    team=sal.findAll('td', {'data-stat':'team_name'})
    pl_sal=[[td.getText() for td in rows[i].findAll('td')] for i in range (len(rows))]
    pl_season=[[th.getText() for th in rows[i].findAll('th')] for i in range (len(rows))]
    pl_team=[[x['href'] for x in team[i].findAll('a', {'href':True})] for i in range(len(team))]

    df_sal=pd.DataFrame(pl_sal,columns=header)
    season=[]
    for x in pl_season:
        for y in x:
           season.append(y) 
    teams=[]
    for x in pl_team:
        for y in x:
            y=re.findall(r'/teams/(.*)/', y)
            for z in y:
                teams.append(z)
    start_yr=[]
    for x in season:
        x=re.findall('\d\d\d\d',x)
        for y in x:
            start_yr.append(int(y))

    end_yr=[]
    for x in season:
        x=re.findall('\d\d\d\d',x)
        for y in x:
            y=int(y)+1
            end_yr.append(y)

    df_sal['Season']=season
    df_sal.drop(df_sal.index[-1], inplace=True)
    df_sal['team_id']=teams
    df_sal['season_start']=start_yr
    df_sal['season_end']=end_yr
    player_id=pd.DataFrame(pl_id, columns=['player_id'])
    df_salary=pd.concat([df_sal, player_id], sort=False)
    df_salary.bfill(inplace=True)
    df_salary.reset_index(inplace=True)
    df_salary.drop(df_salary.index[-1],inplace=True)
    df_salary.reset_index(inplace=True)
    df_salary.drop(['level_0','index'], axis=1, inplace=True)
    return df_salary

In [6]:
#Loop to create CSV Files
for url in salary_url:
    try:
        data = salScraping(url)
        name = 'data/nba_salary/nba_salary_'+str(data['player_id'][0])+'.csv'
        data.to_csv(name)
    except AttributeError:
        print ('No salary for: '+str(url))
    except:
        print ('Error with: '+str(url))
        

No salary for: https://www.basketball-reference.com/players/b/benneel01.html
No salary for: https://www.basketball-reference.com/players/b/boneyde01.html
No salary for: https://www.basketball-reference.com/players/b/bookeme01.html
No salary for: https://www.basketball-reference.com/players/c/carruji01.html
No salary for: https://www.basketball-reference.com/players/d/driggna01.html
No salary for: https://www.basketball-reference.com/players/g/grayev01.html
No salary for: https://www.basketball-reference.com/players/m/marticu01.html
No salary for: https://www.basketball-reference.com/players/s/smithst02.html
No salary for: https://www.basketball-reference.com/players/s/steigma01.html
No salary for: https://www.basketball-reference.com/players/d/djordal01.html
No salary for: https://www.basketball-reference.com/players/h/hansore01.html
No salary for: https://www.basketball-reference.com/players/m/mcdonmi01.html
No salary for: https://www.basketball-reference.com/players/a/alumape01.html


In [135]:
#Concat nba_salary data
file_name='data/nba_salary/nba_salary_{}.csv'
player_id=[x for x in adv_stats['player_id'].unique()]
salary_list=[]
for x in player_id:
    try:
        salary_list.append(pd.read_csv(file_name.format(x)))
    except FileNotFoundError:
        print('No Salary: '+str(x))

salary=pd.concat(salary_list)

No Salary: benneel01
No Salary: boneyde01
No Salary: bookeme01
No Salary: carruji01
No Salary: driggna01
No Salary: grayev01
No Salary: marticu01
No Salary: smithst02
No Salary: steigma01
No Salary: djordal01
No Salary: hansore01
No Salary: mcdonmi01
No Salary: alumape01
No Salary: bakerla01
No Salary: simsal01
No Salary: wheelty01
No Salary: colsose01
No Salary: mcclida01
No Salary: vanteda01
No Salary: fontais01
No Salary: rigauan01
No Salary: cartema01
No Salary: fergude01
No Salary: fullehi01
No Salary: penigde01
No Salary: bakerma01
No Salary: jonesma03
No Salary: knighbr02
No Salary: felixno01
No Salary: grundan01
No Salary: zimmede01
No Salary: pinknke01
No Salary: allrela01
No Salary: powelka01
No Salary: crawfjo01
No Salary: jeffedo01
No Salary: anderan02
No Salary: jeffeot01
No Salary: lafayol01
No Salary: cousima01
No Salary: ewingpa02
No Salary: byarsde01
No Salary: dentmju01
No Salary: dysonje01
No Salary: smithje01
No Salary: ubileed01
No Salary: wrighch02
No Salary: davi

In [146]:
#Convert Salary into float 
salary['Salary']=salary.Salary.str.replace('$','')
salary['Salary']=salary.Salary.str.replace(',','')
salary['clean_Salary']=salary.Salary.str.extract('(\d+)', expand=True)
salary['clean_Salary']=salary.clean_Salary.astype(float)

#Remove NaN clean_Salary. The Salary is either not available or '< Minimum'
salary.dropna(inplace=True)

In [149]:
#Salary and inflation
inflation=pd.read_csv('data/CPIAUCNS.csv', parse_dates=True)

#Convert date to datetime in order to extract year
inflation.DATE=pd.to_datetime(inflation.DATE)
inflation['YEAR']=inflation.DATE.dt.year

#Narrow down inflation to Year and Inflation
inflation=inflation[['YEAR','CPIAUCNS']]

#Create a mean for each year
inflation=inflation.groupby('YEAR').mean()

#Reset Index
inflation=inflation.reset_index()

#Create a multiplier for inflation
inflation['CPIMult']= (inflation.iloc[-1,1]) / (inflation['CPIAUCNS']) 

In [157]:
#Merge salary and inflation to adjust salary
adj_salary=pd.merge(salary, inflation, left_on='season_start', right_on='YEAR')

#Multiply salary by inflation multiplier to get adjusted salary
adj_salary['adj_salary']=(adj_salary.clean_Salary*adj_salary.CPIMult).round()

#Display on certain fields
adj_salary=adj_salary[['player_id', 'adj_salary','Season', 'Team','team_id', 'season_start', 'season_end', 'clean_Salary']]

In [179]:
#Merge adv_stats and adj_salary

df=pd.merge(adv_stats,adj_salary,how='left',left_on=['player_id','Year','Tm'], right_on=['player_id','season_end','team_id'])

In [187]:
#Back Fill NaN. Reason is because some players were traded midway. Missing values are for new teams and total. 
df.bfill(inplace=True)

In [189]:
print(df)

       Unnamed: 0               Player Pos  Age   Tm   G    MP   PER    TS%  \
0               0   Mahmoud Abdul-Rauf  PG   27  SAC  75  2131  13.2  0.524   
1               1  Shareef Abdur-Rahim  PF   20  VAN  80  2802  17.4  0.518   
2               2       Rafael Addison  SF   32  CHH  41   355  10.6  0.476   
3               3       Cory Alexander  PG   23  SAS  80  1454  14.3  0.528   
4               4         Jerome Allen  SG   24  TOT  76   943   8.7  0.451   
5               5         Jerome Allen  SG   24  IND  51   692   8.7  0.489   
6               6         Jerome Allen  SG   24  DEN  25   251   8.9  0.376   
7               7           Ray Allen*  SG   21  MIL  82  2532  14.6  0.541   
8               8       Derrick Alston   C   24  ATL   2    11  -9.0  0.000   
9               9         Ashraf Amaya  PF   25  WSB  31   144   8.8  0.382   
10             10        Greg Anderson   C   32  SAS  82  1659  11.4  0.531   
11             11       Kenny Anderson  PG   26  POR